In [26]:
# CELL 1: ENVIRONMENT SANITY CHECK
!pip list | findstr "langchain langchain-core langchain-huggingface huggingface-hub pinecone-client"

huggingface-hub           0.35.0
langchain                 0.3.27
langchain-community       0.3.29
langchain-core            0.3.76
langchain-huggingface     0.3.1
langchain-openai          0.3.33
langchain-pinecone        0.2.12
langchain-text-splitters  0.3.11


In [27]:
# CELL 2: IMPORTS AND SETUP
import os
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

# Load environment variables from .env file
load_dotenv()
print("✅ Imports and environment loaded successfully.")

✅ Imports and environment loaded successfully.


In [28]:
# CELL 3: INITIALIZE EMBEDDINGS MODEL
try:
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    print("✅ Embeddings model initialized successfully.")
    print(f"Model details: {embeddings.model_name}")
except Exception as e:
    print(f"❌ Error initializing embeddings model: {e}")

✅ Embeddings model initialized successfully.
Model details: sentence-transformers/all-MiniLM-L6-v2


In [29]:
# CELL 5: TEST THE RETRIEVER
try:
    question = "What are the main priorities mentioned in the budget speech?"
    retrieved_docs = retriever.invoke(question)

    print(f"✅ Retriever test successful. Found {len(retrieved_docs)} relevant documents.")
    print("--- Sample Retrieved Content ---")
    for i, doc in enumerate(retrieved_docs):
        print(f"Doc {i+1} (Page {doc.metadata.get('page', 'N/A')}): {doc.page_content[:250]}...")
except Exception as e:
    print(f"❌ Error testing the retriever: {e}")

✅ Retriever test successful. Found 3 relevant documents.
--- Sample Retrieved Content ---
Doc 1 (Page 13.0): 2025 BUDGET SPEECH 
10  |  NATIONAL TREASURY 2025 
 
 
 
This budget also retains the provisional allocations for early retirement, allocations for PRASA and the 
municipal trading entity reforms announced before, but at a slightly lower level than a...
Doc 2 (Page 21.0): 2025 BUDGET SPEECH 
18  |  NATIONAL TREASURY 2025...
Doc 3 (Page 5.0): 2025 BUDGET SPEECH 
2  |  NATIONAL TREASURY 2025 
 
 
 
INTRODUCTION 
Madam Speaker, a national budget is not merely an accounting exercise measuring what we earn, what 
we spend and what we borrow as a nation.  
It is a reflection of the difficult  ...


In [30]:
# CELL 6: USING THE CORRECT CONVERSATIONAL TASK

from huggingface_hub import InferenceClient
import os

# 1. Define the context from our previous retriever test
context = "\n".join([doc.page_content for doc in retrieved_docs])
question = "What are the main priorities mentioned in the budget speech?"

# 2. Create the prompt that will be sent as a user message
# This is the same as before, just formatted as a single string
prompt = f"""
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {question}

Helpful Answer:
"""

# 3. Initialize the official Hugging Face Inference Client
try:
    client = InferenceClient(token=os.getenv("HUGGINGFACEHUB_API_TOKEN"))
    print("✅ Hugging Face Inference Client initialized.")
    
    # 4. Call the model using the CORRECT method: chat_completion
    response = client.chat_completion(
        messages=[{"role": "user", "content": prompt}], # <-- Format the input as a chat message
        model="mistralai/Mistral-7B-Instruct-v0.2",
        max_tokens=512, # Note: parameter is max_tokens for this method
    )

    # 5. Extract the answer from the response object
    answer = response.choices[0].message.content

    print("\n--- FINAL ANSWER ---")
    print(answer)

except Exception as e:
    print(f"❌ Error calling the Hugging Face API directly: {e}")

✅ Hugging Face Inference Client initialized.

--- FINAL ANSWER ---
 The main priorities mentioned in the budget speech are bolstering the state capability needed to deliver quality, reliable and sustainable core services, which is part of pillar three of the economic growth strategy. Despite making additional allocations, there are other long-standing spending pressures that cannot be funded within the current envelope. There was also a commitment to maintaining the current Value Added Tax (VAT) rate of 15%.
